#Import and load dataset

In [246]:
from skimage.transform import resize
from pathlib import Path
import cv2
import numpy as np

In [247]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [248]:
train_path = Path('drive/MyDrive/CS331.M22.KHCL/Dataset/dogscats_small/train')
test_path = Path('drive/MyDrive/CS331.M22.KHCL/Dataset/dogscats_small/test')

In [249]:
def map_labels_to_ints(labels):
  return [1 if label == 'cat' else 0 for label in labels]

In [250]:
X_train = []
y_train = []
for img_path in train_path.iterdir():
    img = cv2.imread(img_path.as_posix())
    img = resize(img, (224,224,3))
    label = img_path.name.split('.')[0]
    X_train.append(img)
    y_train.append(label)

In [251]:
X_test = []
y_test = []
for img_path in test_path.iterdir():
    img = cv2.imread(img_path.as_posix())
    img = resize(img, (224,224,3))
    label = img_path.name.split('.')[0]
    X_test.append(img)
    print(img_path.name)

10.jpg
8.jpg
20.jpg
18.jpg
3.jpg
7.jpg
9.jpg
16.jpg
1.jpg
15.jpg
5.jpg
19.jpg
14.jpg
12.jpg
6.jpg
13.jpg
11.jpg
17.jpg
4.jpg
2.jpg


In [270]:
y_test = ['cat', 'cat', 'cat', 'dog', 'dog', 
          'cat', 'cat', 'cat', 'dog', 'cat', 
          'cat', 'cat', 'cat', 'dog', 'cat', 
          'cat', 'cat', 'dog', 'dog', 'dog']

In [252]:
y_train = map_labels_to_ints(y_train)

In [253]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [254]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [255]:
X_train.shape, y_train.shape

((42, 224, 224, 3), (42,))

#MobileNetv2

In [256]:
base_model = MobileNetV2(include_top=False, input_shape=(224,224,3))

In [257]:
for layer in base_model.layers[:-1]:
    layer.trainable = False

In [258]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2

In [259]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [260]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [261]:
model.fit(X_train, y_train)

2/2 [==============================] - 4s 287ms/step - loss: 0.5148 - accuracy: 0.6429


In [263]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_8
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_pad
13 block_1_depthwise
14 block_1_depthwise_BN
15 block_1_depthwise_relu
16 block_1_project
17 block_1_project_BN
18 block_2_expand
19 block_2_expand_BN
20 block_2_expand_relu
21 block_2_depthwise
22 block_2_depthwise_BN
23 block_2_depthwise_relu
24 block_2_project
25 block_2_project_BN
26 block_2_add
27 block_3_expand
28 block_3_expand_BN
29 block_3_expand_relu
30 block_3_pad
31 block_3_depthwise
32 block_3_depthwise_BN
33 block_3_depthwise_relu
34 block_3_project
35 block_3_project_BN
36 block_4_expand
37 block_4_expand_BN
38 block_4_expand_relu
39 block_4_depthwise
40 block_4_depthwise_BN
41 block_4_depthwise_relu
42 block_4_project
43 block_4_project_BN
44 block_4_add
45 block_5_expand
46 block_5_expand_BN
47 block_5_expand_relu
48 b

In [264]:
for layer in model.layers[:154]:
   layer.trainable = False
for layer in model.layers[154:]:
   layer.trainable = True

In [265]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [266]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
2/2 [==============================] - 4s 284ms/step - loss: 0.2980 - accuracy: 0.9286
Epoch 2/10
2/2 [==============================] - 1s 279ms/step - loss: 0.2951 - accuracy: 0.9286
Epoch 3/10
2/2 [==============================] - 1s 281ms/step - loss: 0.2883 - accuracy: 0.9286
Epoch 4/10
2/2 [==============================] - 1s 279ms/step - loss: 0.2812 - accuracy: 0.9286
Epoch 5/10
2/2 [==============================] - 1s 292ms/step - loss: 0.2733 - accuracy: 0.9286
Epoch 6/10
2/2 [==============================] - 1s 296ms/step - loss: 0.2650 - accuracy: 0.9286
Epoch 7/10
2/2 [==============================] - 1s 286ms/step - loss: 0.2554 - accuracy: 0.9286
Epoch 8/10
2/2 [==============================] - 1s 280ms/step - loss: 0.2480 - accuracy: 0.9286
Epoch 9/10
2/2 [==============================] - 1s 286ms/step - loss: 0.2367 - accuracy: 0.9286
Epoch 10/10
2/2 [==============================] - 1s 284ms/step - loss: 0.2287 - accuracy: 0.9286


In [268]:
y_test = map_labels_to_ints(y_test)
y_test = np.array(y_test)
X_test = np.array(X_test)
X_test.shape, y_test.shape

((20, 224, 224, 3), (20,))

In [269]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 1s/step - loss: 0.0245 - accuracy: 1.0000


[0.024477075785398483, 1.0]